# Indicators Example

> The following is an example of a [Jupyter notebook](https://jupyter.org) - a tutorial of how to retrieve data from the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation) - that illustrates how to use computational content with the [template](https://worldbank.github.io/template). 

## Requirements

In [ ]:
import itertools

import bokeh
import pandas
import requests
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, output_notebook, show

## Retrieve Data

In this example, we retrieve **Population, total** (`SP.POP.TOTL`) from the [World Bank Indicators](https://data.worldbank.org/indicator) for [BRICS](https://infobrics.org).

In [ ]:
url = "https://api.worldbank.org/v2/country/chn;bra;ind;rus;zaf/indicator/SP.POP.TOTL?format=json&per_page=10000"

Let's use [requests](https://requests.readthedocs.io) to send a GET request,

In [ ]:
r = requests.get(url)

Now, let's normalize and create `pandas.DataFrame` from the response,

In [ ]:
# normalize
data = pandas.json_normalize(r.json()[-1])

# create dataframe
df = pandas.DataFrame.from_dict(data)

```{tip}
Alternatively, the World Bank API supports downloading the data as an [archive](http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2000&source=2&downloadformat=csv). 
```

Let's take a look at the dataframe, 

In [ ]:
df = df.pivot_table(values="value", index="date", columns="countryiso3code")
df = df / 1e6 # scaling
df

## Visualization

Let's now plot the data as a time series using [Bokeh](https://docs.bokeh.org).

In [ ]:
output_notebook()

p = figure(title="Population, total (World Bank)", width=700, height=600)

# colors
colors = itertools.cycle(Spectral6)

# plotting the line graph
for column, color in zip(df.columns, colors):
    p.line(
        df.index,
        df[column],
        legend_label=column,
        color=color,
        line_width=2,
    )

p.legend.location = "right"
p.legend.click_policy = "mute"
p.title.text_font_size = "12pt"

p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Population, total (in millions)"

show(p)